In [14]:
import os, sys
from pathlib import Path 
sys.path.append(str(Path().resolve().parent))
from src.dataset.clustering_dataset import ClusteringDataset
import torch

In [12]:

target_dir = r"C:/Users/kotat/MyPrograms/MyKuzushiji/kuzushiji-recognition/char_sep_datas"
test_docs = [
    '200021637',
    '100249371',
    '100249537',
    '200005598',
    '200014740',
    '200020019',
    '200021712',
    '200021869'
]

# データセットとデータローダの作成例
train_dataset = ClusteringDataset(
    root_dir=target_dir,
    canvas_width=2048,
    transform=None,  # ここに正規化などを入れてもよい
    test_mode=True,
    test_docs=test_docs
)

In [15]:
idx = 0
sample = train_dataset.samples[idx]
image_path = sample["image_path"]
image_id = sample["image_id"]
df = sample["df"]

# 画像ロード
img, orig_w, orig_h = train_dataset.get_image(image_path, target_width=train_dataset.canvas_width,patch_size=256)  # (C, H, W) tensor

img = img / 255.0  # 0-1 正規化
# 必要なら変換
if train_dataset.transform is not None:
    img = train_dataset.transform(img)

# アノテーション抽出
ann_df = train_dataset.get_annotation_data(df, image_id)

# キャンバス生成
canvas_main = train_dataset.draw_rects_from_df(ann_df, train_dataset.canvas_width, orig_w, orig_h)
canvas_glue = train_dataset.draw_glue_from_df(ann_df, train_dataset.canvas_width, orig_w, orig_h)
canvas = torch.stack([canvas_main, canvas_glue], dim=0)  # (2, Hc, Wc)
